In [ ]:
import requests
from langchain.tools import tool
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver  # optional but helps
from langgraph.prebuilt import ToolNode
from typing import TypedDict, Annotated, Sequence
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from dotenv import load_dotenv

load_dotenv()

In [ ]:
## Tools

@tool
def add(a: int, b: int):
    """Adds two integer numbers and returns the result"""
    print(f"Tool ADD : {a} {b}")
    return a + b

@tool
def multiply(a: int, b: int):
    """Multiply two intergers and returns the result"""
    print(f"Tool MULT : {a} {b}")
    return a * b

tools = [add, multiply]

In [ ]:
llm = ChatGroq(model="openai/gpt-oss-120B", temperature=0.2).bind_tools(tools)

system_prompt = SystemMessage(content= """
    You are a helpful Math Expert. You will be asked questions in natural language.
    Answer them with the tools in hand. Dont use your own intuition for 
    math calculation. Use the tools only. If the question seems that it can't be answered
    using the available tools, clearly mention "I can't do that".
""")

In [ ]:
class AgentState(MessagesState):
    pass

In [ ]:
def llm_node(state: AgentState) -> AgentState:
    resp = llm.invoke([system_prompt] + state["messages"])
    return AgentState(messages= [resp])

tool_node = ToolNode(tools)

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("llm", llm_node)
graph.add_node("tools", tool_node)

graph.add_edge(START, "llm")
graph.add_conditional_edges(
    "llm", 
    lambda state: "tools_edge" if state["messages"][-1].tool_calls else "end_edge", 
    {
        "tools_edge": "tools",
        "end_edge": END
    }
)
graph.add_edge("tools", "llm")

agent = graph.compile()

In [ ]:
from IPython.display import Image, display
display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
while True:
    question = input("Ask your math question: ")
    print("Query: " + question)
    result = agent.invoke(AgentState(
        messages=[HumanMessage(content=question)]))
    final_message = result["messages"][-1].content
    print("\nFinal Answer:", final_message)